# Build an agent using UC tools and PythonModel
This notebook will guide you through how to utilize MLflow PythonModel with type hints and Unitycatalog functions as tools to build an agent.

## Prerequisite

Install required packages:
```
pip install mlflow==2.20.0 'unitycatalog-langchain[databricks]==0.1.1' langchain_openai==0.3.7
```

Follow the [instruction](https://docs.databricks.com/aws/en/dev-tools/cli/authentication#authentication-for-the-databricks-cli) to authenticate to your Databricks workspace. Alternatively, check the [UnityCatalog client guidance](https://docs.unitycatalog.io/ai/client/#using-the-client-for-agent-tool-calling) on how to use UnityCatalog Server.

In [ ]:
import mlflow

# start the mlflow server with `mlflow server` first, then set the tracking uri
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
mlflow.langchain.autolog()

## Create a model

### Create tools

In [ ]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# sets the default uc function client
set_uc_function_client(client)

In [ ]:
CATALOG = "ml"
SCHEMA = "serena_test"

In [ ]:
# Define a python code execution function
def execute_python_code(code: str) -> str:
    """
    Executes the given python code and returns its stdout.
    Remember the code should print the final result to stdout.

    Args:
      code: Python code to execute. Remember to print the final result to stdout.
    """
    # clint comment is used to disable lint check, you could delete them
    import sys  # clint: disable=lazy-builtin-import
    from io import StringIO  # clint: disable=lazy-builtin-import

    stdout = StringIO()
    sys.stdout = stdout
    exec(code)  # noqa: S102
    return stdout.getvalue()


function_info = client.create_python_function(
    func=execute_python_code, catalog=CATALOG, schema=SCHEMA, replace=True
)
python_execution_function_name = function_info.full_name

In [ ]:
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"
sql_body = f"""CREATE OR REPLACE FUNCTION {translate_function_name}(content STRING COMMENT 'content to translate', language STRING COMMENT 'target language')
RETURNS STRING
COMMENT 'translate the content to target language, currently only english <-> spanish translation is supported'
RETURN SELECT ai_translate(content, language)
"""
client.create_function(sql_function_body=sql_body)

FunctionInfo(browse_only=None, catalog_name='ml', comment='translate the content to target language, currently only english <-> spanish translation is supported', created_at=1741264002930, created_by='serena.ruan@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language=None, external_name=None, full_data_type='STRING', full_name='ml.serena_test.translate', function_id='015ff53c-cdf9-453b-8e19-9a2332ca7b9f', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='content', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='content to translate', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"content","type":"string","nullable":true,"metadata":{"comment":"content to translate"}}', type_precision=0, type_scale=0), FunctionParameterInfo(name='language', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>,

In [ ]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[
        python_execution_function_name,
        translate_function_name,
    ]
)
tools = toolkit.tools

In [ ]:
tools

[UnityCatalogTool(name='ml__serena_test__execute_python_code', description='Executes the given python code and returns its stdout. Remember the code should print the final result to stdout.', args_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.ml__serena_test__execute_python_code__params'>, func=<function UCFunctionToolkit.uc_function_to_langchain_tool.<locals>.func at 0x3152293f0>, uc_function_name='ml.serena_test.execute_python_code', client_config={'warehouse_id': None, 'profile': None}),
 UnityCatalogTool(name='ml__serena_test__translate', description='translate the content to target language, currently only english <-> spanish translation is supported', args_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.ml__serena_test__translate__params'>, func=<function UCFunctionToolkit.uc_function_to_langchain_tool.<locals>.func at 0x315f21a20>, uc_function_name='ml.serena_test.translate', client_config={'warehouse_id': None, 'profile': None})]

### Define langgraph model

In [ ]:
from typing import Literal

from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)


# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", "agent")

app = workflow.compile()

### Test the model first

In [ ]:
final_state = app.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is MLflow? Keep the response concise and reply in Chinese. Try using as many tools as possible",
            }
        ]
    },
)
response = final_state["messages"][-1].content

## Save the model as python file --> agent.py

## Log the model

In [ ]:
input_example = [{"messages": [{"role": "user", "content": "What is DSPy?"}]}]

with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        name="model",
        # Pass the path to the saved model file
        python_model="agent.py",
        input_example=input_example,
    )

2025/03/06 21:08:14 INFO mlflow.models.signature: Inferring model signature from type hints
2025/03/06 21:08:14 INFO mlflow.models.signature: Running the predict function to generate output based on input example
2025/03/06 21:08:52 INFO mlflow.models.model: Found the following environment variables used during model inference: [OPENAI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run angry-conch-727 at: http://127.0.0.1:5000/#/experiments/0/runs/30690b953587448597c77ca9b2cfa371
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


## Load model as pyfunc and predict

In [ ]:
pyfunc_model = mlflow.pyfunc.load_model(model_info.model_uri)
pyfunc_model.predict(input_example)

'DSPy is a framework for building and deploying decision-making systems in Python. It is used primarily in the context of machine learning and data science to create systems that can make predictions or decisions based on input data. DSPy facilitates the definition of decision rules, decision trees, and other logical constructs, allowing data scientists and developers to implement complex decision-making logic in a structured and manageable way.\n\nKey features of DSPy may include:\n\n1. **Declarative Syntax**: Provides a way to define rules and conditions in a readable and maintainable manner.\n2. **Integration with Machine Learning**: Supports the incorporation of predictive models to enhance decision-making processes.\n3. **Scalability**: Built to handle large datasets and complex decision logic without significant performance bottlenecks.\n4. **Modularity**: Allows for reusable components and functions, making it easier to manage and update decision systems.\n\nOverall, DSPy aims t

In [ ]:
mlflow.models.predict(
    model_uri=model_info.model_uri,
    input_data=[{"messages": [{"role": "user", "content": "What's Spanish for hello?"}]}],
    env_manager="uv",
)

2025/03/06 22:00:44 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/03/06 22:00:44 INFO mlflow.utils.virtualenv: Creating a new environment in /tmp/virtualenv_envs/mlflow-fff8396596a3739547c1db4c7f6f29e166a523da with python version 3.10.15 using uv
Using CPython 3.10.15 interpreter at: /Users/serena.ruan/miniconda3/envs/test/bin/python
Creating virtual environment at: /tmp/virtualenv_envs/mlflow-fff8396596a3739547c1db4c7f6f29e166a523da
Activate with: source /tmp/virtualenv_envs/mlflow-fff8396596a3739547c1db4c7f6f29e166a523da/bin/activate
2025/03/06 22:00:46 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.10.15 environment at /tmp/virtualenv_envs/mlflow-fff8396596a3739547c1db4c7f6f29e166a523da
Resolved 3 packages in 1.40s
Prepared 3 packages in 4.53s
Installed 3 packages in 34ms
 + pip==24.2
 + setuptools==75.1.0
 + wheel==0.44.0
Using Python 3.10.15 environment at /tmp/virtualenv_envs/mlflow-fff8396596a3739547c1db4c7f6

{"predictions": "The Spanish word for \"hello\" is \"hola\"."}